# Testeo para Spider HTML

In [80]:
import requests
import re
from bs4 import BeautifulSoup
import time

In [81]:
url = 'https://www.amazon.ae/Ariel-Automatic-Laundry-Detergent-Original/dp/B076L3PYX6/ref=pd_vtp_h_pd_vtp_h_d_sccl_2/257-2861550-0109024?pd_rd_w=Gh3Nt&content-id=amzn1.sym.84eb92b9-87b1-451d-8969-226f5a04c3f5&pf_rd_p=84eb92b9-87b1-451d-8969-226f5a04c3f5&pf_rd_r=TWHGYCDGMJFWJDP009YD&pd_rd_wg=Z3w7T&pd_rd_r=094a09ae-684d-4c15-b779-42bfe86a86a4&pd_rd_i=B076L3PYX6&psc=1'
url = 'https://www.amazon.ae/Pampers-Premium-Absorption-Ultimate-Protection/dp/B07SRSVW9B/ref=sr_1_5?crid=21HM2G837EYZ0&dib=eyJ2IjoiMSJ9.GZ3duzhDOLb79dFm1q9Ip1XWxVh_SBEdckjiR63DX8LhaS8lFX2mGs8-YCrrSIE9Ef5W7iI-dP0wytWqzJHzIK2LfKbEQrLIApnWzkuYJHsawIqMqf0OLOSRiApGGK2uGY6uPkgncxtVZC1e15EF-f0AgZc9AeKtJHOPqPfh5w1er9tcgJMDRZ-UnWbBkzSsSNDnX6b-QR3igoHMS1jAmmBv7riR9NiX_DX-ezYsXfUdfk9jMi_r5NKWDYBC2N59H44UsqDwegUqSVPazFa98Vd0qWY95NQOXw2iYVkINZg.cAbZZvD9Un4MEV6C9bPFHztPonLU_GC9qbiEuA_UZDw&dib_tag=se&keywords=pampers&qid=1712775725&sprefix=pamper%2Caps%2C376&sr=8-5&th=1'

In [82]:
payload = {}
headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
  'Accept-Language': 'en-US;q=0.5,en;q=0.3',
  'Accept-Encoding': 'gzip, deflate, br',
  'Alt-Used': 'www.amazon.ae',
  'Connection': 'keep-alive',
  'Upgrade-Insecure-Requests': '1',
  'Sec-Fetch-Dest': 'document',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-Site': 'none',
  'Sec-Fetch-User': '?1',
  'Sec-GPC': '1'
}

In [83]:
response = requests.request("GET", url, headers=headers, data=payload)
html_content = response.text
soup = BeautifulSoup(html_content, "html.parser")

In [84]:
title = soup.find('span', id='productTitle').text.strip() if soup.find('span', id='productTitle') != None else 'No title'
print(f"---- Title:{title}")

---- Title:Pampers Premium Care Newborn Taped Diapers, Size 2, 3-8kg, Unique Softest Absorption for Ultimate Skin Protection, 168 Count


In [85]:
buy_box = soup.select_one('#merchantInfoFeature_feature_div > div:nth-child(2) > div:nth-child(1) > span:nth-child(1) > a:nth-child(1)').text if soup.select_one('#merchantInfoFeature_feature_div > div:nth-child(2) > div:nth-child(1) > span:nth-child(1) > a:nth-child(1)') != None else ''
if buy_box == '':
    buy_box = soup.select_one('#merchantInfoFeature_feature_div > div:nth-child(2) > div:nth-child(1) > span:nth-child(1)').text if soup.select_one('#merchantInfoFeature_feature_div > div:nth-child(2) > div:nth-child(1) > span:nth-child(1)') != None else ''
if buy_box == '':
    buy_box = soup.select_one('tr.a-spacing-micro:nth-child(2) > td:nth-child(2) > span:nth-child(1) > a:nth-child(1)').text if soup.select_one('tr.a-spacing-micro:nth-child(2) > td:nth-child(2) > span:nth-child(1) > a:nth-child(1)') != None else ''

In [86]:
if buy_box == '' or 'Amazon.ae' not in buy_box:
    # Check if the store is transmed
    others_offer = soup.find_all('span', class_='a-size-small mbcMerchantName')# Catura para testear que lleguen los nombres
    others_offer_container = soup.find_all('div', class_='a-box-inner a-padding-small')  # Assuming class names remain the same

    # Print others_offer for debugging purposes (if needed)
    print(f"OTHERS OFFER:{[o.text.strip() for o in others_offer]}\n")

    for container in others_offer_container:
        title_store = container.find('span', class_='a-size-small mbcMerchantName').text if container.find('span', class_='a-size-small mbcMerchantName') != None else ''
        if title_store and 'Amazon.ae' in title_store.text.strip():
            final_price_element = container.find('span', class_='a-size-medium a-color-price')
            if final_price_element:
                final_price = fc.just_price(price_str=final_price_element.text.strip(), dotted_price=False)
                is_transmed = True
                break  # Exit the loop after finding the first Transmed offer

In [87]:
others_string = ''
others_offer_container = soup.find_all('div', class_='a-box-inner a-padding-small')
for i in range(1, len(others_offer_container)):
        title_store = others_offer_container[i].find('span', class_='a-size-small mbcMerchantName').text if others_offer_container[i].find('span', class_='a-size-small mbcMerchantName') != None else ''
        store_price = others_offer_container[i].select_one('span.a-size-medium.a-color-price').text if others_offer_container[i].select_one('span.a-size-medium.a-color-price') != None else 0
        others_string += f"Store:{title_store.strip()} Price:{store_price.strip()}, "

In [88]:
print(others_string)

In [89]:
final_price = 0
final_price_int = soup.select_one("span.a-price.aok-align-center.reinventPricePriceToPayMargin.priceToPay span span.a-price-whole").text if soup.select_one("span.a-price.aok-align-center.reinventPricePriceToPayMargin.priceToPay span span.a-price-whole") != None else ''
final_price_decimals = soup.select_one("span.a-price.aok-align-center.reinventPricePriceToPayMargin.priceToPay span span.a-price-fraction").text if soup.select_one("span.a-price.aok-align-center.reinventPricePriceToPayMargin.priceToPay span span.a-price-fraction") != None else ''
if final_price_int != None and final_price_int != '':
    final_price_int = re.sub("[^0-9.]", "", final_price_int)
    final_price_int = re.sub("[ ]", "", final_price_int)
    final_price_int = re.sub("[.]", "", final_price_int)

if final_price_decimals != None and final_price_decimals != '':
    final_price_decimals = re.sub("[^0-9.]", "", final_price_decimals)
    final_price_decimals = re.sub("[ ]", "", final_price_decimals)
    final_price_decimals = re.sub("[.]", "", final_price_decimals)

if final_price_int != None and final_price_int != '' and final_price_decimals != None and final_price_decimals != '':
    try:
        final_price = f"{final_price_int}.{final_price_decimals}"
        final_price = float(final_price)
    except:
        final_price = 0
        print(f"1:No se pudo convertir a flotante final_price:{final_price}")

print(f"FinalPrice: {final_price}")

FinalPrice: 0


In [90]:
if final_price == 0:
    final_price = soup.select_one("#newAccordionRow_0 > div:nth-child(1) > div:nth-child(1) > h5:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > span:nth-child(1) > span:nth-child(1)").text if soup.select_one("#newAccordionRow_0 > div:nth-child(1) > div:nth-child(1) > h5:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > span:nth-child(1) > span:nth-child(1)") != None else 0 
    # final_price = fcs.just_price(price_str=final_price, dotted_price=False)

In [91]:
old_price = soup.select_one(".basisPrice > span:nth-child(1) > span:nth-child(1)").text if soup.select_one(".basisPrice > span:nth-child(1) > span:nth-child(1)") != None else 0
print(f"OldPrice:{old_price}")

OldPrice:0


In [92]:
if old_price == 0 or old_price == '':
    old_price = soup.select_one("td.a-color-secondary:nth-child(2) > span:nth-child(1) > span:nth-child(1)").text if soup.select_one("td.a-color-secondary:nth-child(2) > span:nth-child(1) > span:nth-child(1)") != None else 0


In [93]:
bullet_points_list = soup.select("li.a-spacing-mini")
bullet_points = ''

try:
    for i in range(0, len(bullet_points_list)):
        content = bullet_points_list[i].text.strip()
        if i == len(bullet_points_list) - 1:
            bullet_points += f"•{content}."
        else:
            bullet_points += f"•{content}, "
except:
    print(f"No se pudo extraer los bullets points")
print(f"BulletPoint: {bullet_points}")

BulletPoint: •ULTIMATE SKIN PROTECTION & UNIQUE SOFTEST ABSORPTION: Pampers Premium Care Newborn Taped Diapers come with cotton-like softness and built-in lotion to ensure unique softest absorption for ultimate skin protection., •SOFTEST DIAPER IN THE GCC: Pampers Premium Care Baby Diapers provide all round soft and stretchy sides, gently wrapping your baby in silky softness to give an extra comfortable fit. They protect your baby’s delicate belly and comfortably snug their body. No wonder Pampers Premium Care Diapers are the best diapers in the GCC., •NAVEL CUT: Specifically designed for newborns, Pampers Baby Diapers Size 2 come with navel cut that will protect your baby's belly with a perfectly contoured fit to always ensure their comfort., •BREATHABLE MATERIALS: These diapers are made with a lotion that helps protect your baby from skin irritation. The breathable materials with micropores let fresh air in and humid air out allowing the skin to breathe freely., •RUNNY-MESS ABSORPTIO

In [94]:
description = soup.find(id='productDescription').text.strip() if soup.find(id='productDescription') != None else ''
print(f"Description:{description}")

Description:Product Description   Pampers Premium Care Newborn Baby Taped Diaper is soft and gentle on your little one’s skin. It offers unique softest absorption and ultimate skin protection to ensure your baby will have a leak-free comfort. Designed in German labs, Pampers Premium Care Diapers have been voted the softest diapers in the GCC by moms. Their cotton-like softness and built-in lotion keep your baby’s skin healthy and soft while helping prevent skin irritation. What is the difference between Pampers and Pampers Premium Care? Pampers Premium Care Baby Diapers come with a special layer to ensure extra absorption and prevent runny or messiness. This extra layer helps distribute wetness evenly, thereby ensuring dryness and less bulk. Which age can Pampers Premium Care diapers be used for? Pampers Size 2 Premium Care Newborn Taped Diapers are designed to provide comfortable and ultimate skin protection care to babies weighing from 3-8kg, making them a must for newborn baby items

In [95]:
q_images_list = soup.find_all("li", class_="a-spacing-small")
q_images = 0
for i in q_images_list:
    content = i.find('img')["src"] if i.find('img') != None else ''
    if 'https://m.media-amazon.com/' in content and '.jpg' in content:
        q_images += 1
print(q_images - 1)

6


In [96]:
a_plus = 1 if soup.find_all('div', id='aplus') != [] else 0
print(f"{a_plus}")

1


In [97]:
best_sellers_rank_elements = soup.find("div", id="productDetails_db_sections").div.table.find_all("tr") if soup.find("div", id="productDetails_db_sections") != None else ''

In [98]:
try:
    for i in best_sellers_rank_elements:
        content = i.text
        if "Best Sellers Rank" in content:
            ranks = re.findall(r"#(\d+)", content)
            category_best_sellers_rank = int(ranks[0])
            catgroup_best_sellers_rank = int(ranks[1])
except Exception as e:
    print(f"ERROS - {e}")

In [99]:
print(f"category_best_sellers_rank:{category_best_sellers_rank}")
print(f"catgroup_best_sellers_rank:{catgroup_best_sellers_rank}")

category_best_sellers_rank:4
catgroup_best_sellers_rank:1


In [100]:
rating = soup.select("#acrPopover > span.a-declarative > a > i")[0].text if soup.select("#acrPopover > span.a-declarative > a > i") != [] else 0
rating = rating.replace("out of 5 stars", "").strip()
rating = float(rating)
print(f"Rating:{rating}")

Rating:4.5


In [101]:
q_rating = soup.select("#acrCustomerReviewText")[0].text if soup.select("#acrCustomerReviewText") != [] else 0
print(f"QRating:{q_rating}")

QRating:1,950


In [102]:
try:
    q_rating = q_rating.replace("ratings", "").strip()
    q_rating = int(q_rating)
except:
    q_rating = 0
print(f"QRating:{q_rating}")

QRating:0


In [104]:
others_list = soup.find("div", id="aod-offer-price")
print(f"OthersList:{others_list}")

OthersList:None
